In [1]:
from  Controller import TrainingController_V2, ExplainingController_V2
from Parameters.PredictingParameters import PredictingParameters
from Parameters.TrainingParameters import TrainingParameters
from Parameters.Enums import SelectableLoss, SelectableModels, SelectableDatasets
import torch
import pyAgrum.lib.notebook as gnb
from IPython.core.display import display, HTML

/Users/jrhs/.pyenv/versions/3.9.4/envs/dl/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
model_path = "./SavedModels/0.7237_Diabetes_BaseNNModel_2021-05-12 04:17:18.144205"
loss_fn = SelectableLoss.BCE
model=SelectableModels.BaseNNModel
dataset = SelectableDatasets.Diabetes

In [3]:
########## Setting up training parameters ##########
TrainingParameters.load_model_folder_path = model_path
TrainingParameters.loss = loss_fn
TrainingParameters.model = model
TrainingParameters.dataset = dataset

In [4]:
trainer = TrainingController_V2()


| Running on cpu  

| Model loaded successfully from: ./SavedModels/0.7237_Diabetes_BaseNNModel_2021-05-12 04:17:18.144205  


<Figure size 1600x800 with 0 Axes>

In [5]:
######## Setting up predicting parameters ########
PredictingParameters.load_model_folder_path = model_path
PredictingParameters.loss = loss_fn

In [6]:
explainer = ExplainingController_V2()


| Model loaded successfully from ./SavedModels/0.7237_Diabetes_BaseNNModel_2021-05-12 04:17:18.144205/model.pt 


In [7]:
############ Get example from test set ############
example_idx = 5
example_data = torch.tensor(trainer.test_dataset[example_idx][trainer.feature_names]).unsqueeze(0).float().to(explainer.device)

In [8]:
############ Generate examplaination ############
cat_df, predicted_value, bn, bn_html, inference, infoBN, markov_blanket_html= explainer.medical_predict_lindaBN_explain(example_data, 500, variance=1)


| Selecting Greedy Hill Climbing Algorithm 
/Users/jrhs/.pyenv/versions/3.9.4/envs/dl/lib/python3.9/site-packages/torch/distributions/distribution.py:151: UserWarning: sample_n will be deprecated. Use .sample((n,)) instead
  warnings.warn('sample_n will be deprecated. Use .sample((n,)) instead', UserWarning)


In [9]:
true_inference = gnb.getInference(
            bn, evs={explainer.target_name: "True"}, targets=cat_df.columns.values, size="70")

In [10]:
display(HTML(true_inference))